![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Lab | Making predictions with logistic regression

In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.



**1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.**

In [1]:
# prep: import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
import numpy as np
password = getpass.getpass()

········


In [2]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [3]:
query_features = '''SELECT f.title AS 'film_title', f.rental_rate,  f.rental_duration, f.length, f.rating,  f.special_features, c.name AS category
FROM sakila.film as f
JOIN sakila.film_category fc USING (film_id)
JOIN sakila.category c USING (category_id)
ORDER BY title;'''
features = pd.read_sql_query(query_features, engine)

In [4]:
features

,film_title,rental_rate,rental_duration,length,rating,special_features,category
0,ACADEMY DINOSAUR,0.99,6,86,PG,"Deleted Scenes,Behind the Scenes",Documentary
1,ACE GOLDFINGER,4.99,3,48,G,"Trailers,Deleted Scenes",Horror
2,ADAPTATION HOLES,2.99,7,50,NC-17,"Trailers,Deleted Scenes",Documentary
3,AFFAIR PREJUDICE,2.99,5,117,G,"Commentaries,Behind the Scenes",Horror
4,AFRICAN EGG,2.99,6,130,G,Deleted Scenes,Family
...,...,...,...,...,...,...,...
995,YOUNG LANGUAGE,0.99,6,183,G,"Trailers,Behind the Scenes",Documentary
996,YOUTH KICK,0.99,4,179,NC-17,"Trailers,Behind the Scenes",Music
997,ZHIVAGO CORE,0.99,6,105,NC-17,Deleted Scenes,Horror
998,ZOOLANDER FICTION,2.99,5,101,R,"Trailers,Deleted Scenes",Children


**2. Create a query to get the list of films and a boolean indicating if it was rented last month (May 2005). This would be our target variable.**

In [5]:
query_target = '''SELECT f.title AS 'film_title'
FROM sakila.film f 
JOIN sakila.inventory i USING (film_id)
JOIN sakila.rental r USING (inventory_id)
WHERE r.rental_date LIKE '%%2005-05%%'
GROUP BY f.title
ORDER BY f.title;'''
target = pd.read_sql_query(query_target, engine)

In [6]:
target

,film_title
0,ACADEMY DINOSAUR
1,ADAPTATION HOLES
2,AFFAIR PREJUDICE
3,AFRICAN EGG
4,AGENT TRUMAN
...,...
681,WYOMING STORM
682,YENTL IDAHO
683,ZHIVAGO CORE
684,ZOOLANDER FICTION


In [7]:
# Lets add a 1 for all the films rented in May
target['rented_in_may']=1

In [8]:
target

,film_title,rented_in_may
0,ACADEMY DINOSAUR,1
1,ADAPTATION HOLES,1
2,AFFAIR PREJUDICE,1
3,AFRICAN EGG,1
4,AGENT TRUMAN,1
...,...,...
681,WYOMING STORM,1
682,YENTL IDAHO,1
683,ZHIVAGO CORE,1
684,ZOOLANDER FICTION,1


In [9]:
#target=target.drop(['rental_date'],axis=1)

**3. Read the data into a Pandas dataframe.**


First of all I want to have the same number of rows in each Data frame, therefore I am going to compare the missiong film_id in target (those not rented in may), and I am going to add them

In [10]:
film_title_target = list(features['film_title'])
film_title_features = list(target['film_title'])

In [11]:
len(film_title_target)

1000

In [12]:
len(film_title_features)

686

In [13]:
# Compare the two lists and append the different values into a new list
films_not_rented_in_may = [x for x in film_title_target if x not in film_title_features]

In [14]:
len(films_not_rented_in_may) # 1000 - 686 = 314; as expected

314

In [15]:
# Lets convert the list into a boolean column so we can lated add it to target
df_films_not_rented_in_may=pd.DataFrame(films_not_rented_in_may)
df_films_not_rented_in_may['rented_in_may']=0
df_films_not_rented_in_may

,0,rented_in_may
0,ACE GOLDFINGER,0
1,AIRPLANE SIERRA,0
2,ALABAMA DEVIL,0
3,ALADDIN CALENDAR,0
4,ALI FOREVER,0
...,...,...
309,WORKER TARZAN,0
310,WORLD LEATHERNECKS,0
311,WRONG BEHAVIOR,0
312,YOUNG LANGUAGE,0


In [16]:
# Rename the 0 column for its actual meaning (film_title)
df_films_not_rented_in_may=df_films_not_rented_in_may.rename({0:'film_title'},axis=1)

In [17]:
df_films_not_rented_in_may

,film_title,rented_in_may
0,ACE GOLDFINGER,0
1,AIRPLANE SIERRA,0
2,ALABAMA DEVIL,0
3,ALADDIN CALENDAR,0
4,ALI FOREVER,0
...,...,...
309,WORKER TARZAN,0
310,WORLD LEATHERNECKS,0
311,WRONG BEHAVIOR,0
312,YOUNG LANGUAGE,0


In [18]:
# We add to our target data frame our not rented in May films
target= pd.concat([target,df_films_not_rented_in_may], ignore_index=True)

In [19]:
target

,film_title,rented_in_may
0,ACADEMY DINOSAUR,1
1,ADAPTATION HOLES,1
2,AFFAIR PREJUDICE,1
3,AFRICAN EGG,1
4,AGENT TRUMAN,1
...,...,...
995,WORKER TARZAN,0
996,WORLD LEATHERNECKS,0
997,WRONG BEHAVIOR,0
998,YOUNG LANGUAGE,0


In [20]:
# Order by alphabetically and reseting the index
target = target.sort_values('film_title').reset_index()
target = target.drop(['index'],axis=1)

In [21]:
target

,film_title,rented_in_may
0,ACADEMY DINOSAUR,1
1,ACE GOLDFINGER,0
2,ADAPTATION HOLES,1
3,AFFAIR PREJUDICE,1
4,AFRICAN EGG,1
...,...,...
995,YOUNG LANGUAGE,0
996,YOUTH KICK,0
997,ZHIVAGO CORE,1
998,ZOOLANDER FICTION,1


In [22]:
# Lets drop the common column of features and target: film_title
target = target.drop(['film_title'],axis=1)

Both data Frames are uniformed and ready to be concatenated

In [23]:
data= pd.concat([features,target], axis=1)
data

,film_title,rental_rate,rental_duration,length,rating,special_features,category,rented_in_may
0,ACADEMY DINOSAUR,0.99,6,86,PG,"Deleted Scenes,Behind the Scenes",Documentary,1
1,ACE GOLDFINGER,4.99,3,48,G,"Trailers,Deleted Scenes",Horror,0
2,ADAPTATION HOLES,2.99,7,50,NC-17,"Trailers,Deleted Scenes",Documentary,1
3,AFFAIR PREJUDICE,2.99,5,117,G,"Commentaries,Behind the Scenes",Horror,1
4,AFRICAN EGG,2.99,6,130,G,Deleted Scenes,Family,1
...,...,...,...,...,...,...,...,...
995,YOUNG LANGUAGE,0.99,6,183,G,"Trailers,Behind the Scenes",Documentary,0
996,YOUTH KICK,0.99,4,179,NC-17,"Trailers,Behind the Scenes",Music,0
997,ZHIVAGO CORE,0.99,6,105,NC-17,Deleted Scenes,Horror,1
998,ZOOLANDER FICTION,2.99,5,101,R,"Trailers,Deleted Scenes",Children,1


**4. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.**


We have already encode 'rented_in_may', the rest of the features will be treated in the after the X-y split and train, test, split, so the testing sample does not get any transformation

In [24]:
data.isna().sum() 

film_title          0
rental_rate         0
rental_duration     0
length              0
rating              0
special_features    0
category            0
rented_in_may       0
dtype: int64

In [25]:
# We don't want to drop the films not ranted in may obvisuly, but we want to fill them with 0
data=data.fillna(0)

In [26]:
data.isna().sum() 

film_title          0
rental_rate         0
rental_duration     0
length              0
rating              0
special_features    0
category            0
rented_in_may       0
dtype: int64

In [27]:
data.shape

(1000, 8)

**5. Create a logistic regression model to predict this variable from the cleaned data.**


X-y Split

In [28]:
y = data['rented_in_may']
X = data.drop(['rented_in_may'], axis = 1)

Train, test, split

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) # default fraction is .2

In [31]:
display(X_train.shape)
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

(750, 7)

(250, 7)

(750,)

(250,)

Split between numerical and categorical in train and test set

In [32]:
X_train_num = X_train.select_dtypes(np.number)
X_train_cat = X_train.select_dtypes(object)

In [33]:
X_test_num = X_test.select_dtypes(np.number)
X_test_cat = X_test.select_dtypes(object)

Treating the categorical values (ERINS IDEA)

In [34]:
X_train_cat.special_features = X_train_cat.special_features.str.split(',')

In [35]:
X_test_cat.special_features = X_test_cat.special_features.str.split(',')

In [36]:
pd.Series(['a|b|c','a|d|e','b|c|e']).str.get_dummies()

,a,b,c,d,e
0,1,1,1,0,0
1,1,0,0,1,1
2,0,1,1,0,1


In [37]:
X_train_cat = X_train_cat.drop('special_features',1).join(X_train_cat.special_features.str.join('|').str.get_dummies())

C:\Users\Jose María\AppData\Local\Temp\ipykernel_19380\3289135521.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_train_cat = X_train_cat.drop('special_features',1).join(X_train_cat.special_features.str.join('|').str.get_dummies())


In [38]:
X_test_cat = X_test_cat.drop('special_features',1).join(X_test_cat.special_features.str.join('|').str.get_dummies())

C:\Users\Jose María\AppData\Local\Temp\ipykernel_19380\2753003235.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_test_cat = X_test_cat.drop('special_features',1).join(X_test_cat.special_features.str.join('|').str.get_dummies())


In [39]:
r = pd.get_dummies(X_train_cat.rating, prefix='rating')
r2 = pd.get_dummies(X_test_cat.rating, prefix='rating')

In [40]:
c = pd.get_dummies(X_train_cat.category, prefix='genre')
c2 = pd.get_dummies(X_test_cat.category, prefix='genre')

In [41]:
train_encoded = pd.concat([c,r], axis=1)
test_encoded = pd.concat([c2,r2], axis=1)

In [42]:
X_train_cat = pd.concat([X_train_cat, train_encoded], axis=1)
X_test_cat = pd.concat([X_test_cat, test_encoded], axis=1)

Drop the encoded categories

In [43]:
X_train_cat = X_train_cat.drop(['rating','category','film_title'],axis=1)
X_train_cat

,Behind the Scenes,Commentaries,Deleted Scenes,Trailers,genre_Action,genre_Animation,genre_Children,genre_Classics,genre_Comedy,genre_Documentary,...,genre_Music,genre_New,genre_Sci-Fi,genre_Sports,genre_Travel,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R
253,1,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
667,1,1,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
85,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
969,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
75,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
192,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
629,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
559,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [44]:
X_test_cat = X_test_cat.drop(['rating','category','film_title'],axis=1)
X_test_cat

,Behind the Scenes,Commentaries,Deleted Scenes,Trailers,genre_Action,genre_Animation,genre_Children,genre_Classics,genre_Comedy,genre_Documentary,...,genre_Music,genre_New,genre_Sci-Fi,genre_Sports,genre_Travel,rating_G,rating_NC-17,rating_PG,rating_PG-13,rating_R
993,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
859,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
298,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
553,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
672,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,0,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
356,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
478,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [45]:
from sklearn.preprocessing import MinMaxScaler

In [46]:
transformer = MinMaxScaler().fit(X_train_num)
X_train_norm = transformer.transform(X_train_num)
X_train_num_scale = pd.DataFrame(X_train_norm, index = X_train_num.index, columns=X_train_num.columns)
X_train_num_scale.head()

,rental_rate,rental_duration,length
253,0.5,0.25,0.812950
667,1.0,1.00,0.244604
85,1.0,0.75,0.539568
969,0.0,1.00,0.043165
75,0.0,0.25,0.410072


In [47]:
X_test_norm = transformer.transform(X_test_num)
X_test_num_scale = pd.DataFrame(X_test_norm, index = X_test_num.index, columns=X_test_num.columns)
X_test_num_scale.head()

,rental_rate,rental_duration,length
993,1.0,0.75,0.388489
859,1.0,0.25,0.338129
298,0.0,0.25,0.705036
553,0.5,0.75,0.812950
672,0.0,0.00,0.517986


In [48]:
X_train = pd.concat([X_train_num_scale,X_train_cat], axis = 1)
X_test = pd.concat([X_test_num_scale,X_test_cat], axis = 1)

**6. Evaluate the results.**

In [49]:
from sklearn.linear_model import LogisticRegression


In [50]:
logreg = LogisticRegression().fit(X_train, y_train)
logreg.score(X_train,y_train)

0.6946666666666667